In [1]:
import pandas as pd

data = pd.read_csv('Expanded_Categorical_Values.csv')

In [2]:
data

,Category,Item,Quantity,Comments
0,Fruit,Apple,10,Fresh and juicy
1,Vegetable,Carrot,15,Organic
2,Fruit,Banana,12,Ripe and ready to eat
3,Grain,Rice,20,Basmati
4,Dairy,Milk,8,Whole milk
5,Meat,Chicken,5,Free-range
6,Fruit,Orange,14,Sweet and tangy
7,Vegetable,Spinach,18,Leafy green
8,Grain,Bread,25,Whole grain
9,Dairy,Cheese,9,Aged cheddar


In [4]:
from sentence_transformers import SentenceTransformer 

encoder = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\Projects\NLP_Streamlit\pythonProject1\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prati\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
vectors = encoder.encode(data.Comments)

In [6]:
vectors.shape

(10, 768)

In [8]:
dim = vectors.shape[1]

In [13]:
import faiss
import numpy as np

In [9]:
index = faiss.IndexFlatL2(dim)

In [10]:
index.add(vectors)

In [14]:
search_query = "Basmati"
encode_search_query = np.array(encoder.encode(search_query)).reshape(1,-1)
encode_search_query.shape

(1, 768)

In [17]:
distance, my_index = index.search(encode_search_query,k=2)

In [20]:
data.loc[my_index[0]]

,Category,Item,Quantity,Comments
3,Grain,Rice,20,Basmati
8,Grain,Bread,25,Whole grain


In [1]:
import os

import pickle
import time
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [2]:
model_name = "t5-small"  # You can replace this with a more powerful model like "gpt2" or "EleutherAI/gpt-neo-2.7B"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

D:\Projects\NLP_Streamlit\pythonProject1\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

D:\Projects\NLP_Streamlit\pythonProject1\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prati\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]